# 1.creator类  
creator允许用户创建自己的类  
比如说，对于一个最小化类FitnessMin和个体类Individual  
可以通过creator.create（名字，父类，属性）函数创建

In [10]:
from deap import base, creator
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

# 2.base.toolbox类
作为initializer来创建个体以及种群的实例<br />
通常需要import tools以及 random<br />
首先通过toolbox=base.ToolBox()创建toolbox的实例<br />
接下来使用toolbox.register（函数名，创建方法？，生成类型，属性？，个数）创建函数<br />
创建完以后可以直接使用toolbox.funcname（）创建种群、个体等<br />
例题中的个体是长度为10的随机数!<br />

In [11]:
import random
from deap import tools

IND_SIZE = 10

toolbox = base.Toolbox()
toolbox.register("attribute", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attribute, n=IND_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# 3.创建评价函数evaluate来评价个体优劣
创建evaluate函数、选择合适的评价标准、并且将创建的evaluate函数注册到toolbox里<br />
接下来注册环境选择中的几种函数例如交配（mate），变异（mutate），选择（select），这些函数通常在tools类中包含有不同的方法<br />

In [12]:
def evaluate(individual):
    return sum(individual),

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

# 4.在main函数中创建主程序 

程序结构<br />
1.创建种群<br />
2.设置参数（变异率、交叉率、迭代次数等）<br />
3.使用map函数评价种群中的所有个体<br />
注：map（被运用的函数，被运用的对象<br />
4.迭代评估个体适应度<br />
5.从种群中select下一代个体<br />
6.使用toolbox.clone克隆后代<br />
7.交叉变异<br />
8.选出来发生变化的个体<br />
9.重新评估他们的适应度<br />
10.将种群变量pop替换为offspring<br />

In [14]:
def main():
    pop = toolbox.population(n=50)
    CXPB, MUTPB, NGEN = 0.5, 0.2, 40

    # Evaluate the entire population
    fitnesses = map(toolbox.evaluate, pop)
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    for g in range(NGEN):
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # The population is entirely replaced by the offspring
        pop[:] = offspring

    return pop
main()

[[-1.8434146198077044,
  -2.365663922946479,
  -2.5565930497931597,
  -5.76978010119084,
  0.03701648154115378,
  -3.35994233846423,
  -5.057642541039401,
  -1.9403583831887252,
  -3.341683123488766,
  -4.110677800765874],
 [-1.8434146198077044,
  -2.365663922946479,
  -2.5565930497931597,
  -5.76978010119084,
  0.03701648154115378,
  -3.35994233846423,
  -4.288730718915723,
  -0.8794217843292251,
  -3.341683123488766,
  -4.110677800765874],
 [-1.8434146198077044,
  -2.365663922946479,
  -2.5565930497931597,
  -6.54720108485876,
  0.03701648154115378,
  -2.274848165701507,
  -4.288730718915723,
  -0.8794217843292251,
  -4.008942985999528,
  -4.110677800765874],
 [-3.271551781388567,
  -2.365663922946479,
  -2.5565930497931597,
  -5.76978010119084,
  0.03701648154115378,
  -3.35994233846423,
  -4.288730718915723,
  -0.8794217843292251,
  -4.008942985999528,
  -3.177668195445052],
 [-1.8434146198077044,
  -2.365663922946479,
  -2.5565930497931597,
  -5.76978010119084,
  0.037016481541153